# Testing

In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [36]:
data = pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/app_usage_time_motorola.csv',
            parse_dates=['date'])
data.head(2)

,date,Amazon Shopping,Android System,Biblioapp UdeA,Bing,Calculator,Calculator_1,Calendar,Call Management,Camera,...,Twitch,Twitter,tyba,Upwork Talent,VpnDialogs,Wallpapers,WhatsApp,YouTube,YouTube Music,Zoom
0,2022-10-19,0.0,0.07,0.0,0.0,3.02,0.0,1.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,73.10,0.00,0.0,0.0
1,2022-10-20,0.0,0.07,0.0,0.0,1.57,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,61.83,12.98,0.0,0.0


In [3]:
def transform_dataframe(dataframe: pd.DataFrame, values_greater_than):
    data_copy = dataframe.copy()

    #packing the name of columns and sums in a dictionary to use in further codes
    dictionary_sums = {}
    for column in data_copy:
        if (column != 'date') and \
           (sum(data_copy[column]) > values_greater_than):
            
            dictionary_sums[column] = sum(data_copy[column])

    names_of_columns_to_stay = [column_name for column_name in dictionary_sums.keys()]
    names_of_columns_to_stay.insert(0, 'date')

    #filtering dataframe
    transformed_dataframe = data_copy[names_of_columns_to_stay]
    
    return transformed_dataframe

In [24]:
transform_dataframe(data, 100).head(2)

,date,youtube.com,learn.deeplearning.ai,netflix.com,upwork.com,meet.google.com,wolframalpha.com,web.whatsapp.com,mail.google.com,udeaeduco-my.sharepoint.com,bing.com,github.com,docs.google.com,chat.openai.com,drive.google.com,coursera.org,kaggle.com,instagram.com,colab.research.google.com
0,2022-10-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-10-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Transformation of the actual data

In [4]:
path = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed"
dir_list = listdir(path)
dir_list

['.gitkeep',
 'app_usage_count_motorola.csv',
 'app_usage_time_motorola.csv',
 'device_unlocks.csv',
 'web_usage_count_chrome.csv',
 'web_usage_count_edge.csv',
 'web_usage_count_motorola.csv',
 'web_usage_time_chrome.csv',
 'web_usage_time_edge.csv',
 'web_usage_time_motorola.csv']

In [51]:
import re
# txt = dir_list[2]
# findall("time", txt)

word_to_find = "time"
for file in dir_list:
    word_is_in = re.search(word_to_find, file)
    if word_is_in:
        file_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/{file}'
        data = pd.read_csv(file_path, parse_dates=['date'])
        data_transformed = transform_dataframe(data, 100)

        export_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/database/{file}'
        data_transformed.to_csv(export_path, 
                                sep=';',
                                index=None,
                                date_format='%Y-%m-%d')

In [50]:
file_path

'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/web_usage_time_motorola.csv'

In [44]:
print(word_is_in)

None


In [53]:
pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/database/app_usage_time_motorola.csv', delimiter=';')

,date,Amazon Shopping,Calculator,Calendar,Camera_2,Chrome,Clock,com.activision.callofduty.shooter,Discord,Drive,...,Telegram,TikTok,Truecaller,Twitter,tyba,Upwork Talent,WhatsApp,YouTube,YouTube Music,Zoom
0,2022-10-19,0.00,3.02,1.50,0.25,17.48,1.75,0.00,0.0,55.68,...,0.0,37.42,0.00,0.00,0.00,0.00,73.10,0.00,0.00,0.0
1,2022-10-20,0.00,1.57,0.00,8.23,5.28,0.90,19.43,0.0,11.72,...,0.0,87.10,0.02,0.00,0.00,0.00,61.83,12.98,0.00,0.0
2,2022-10-21,0.00,0.00,0.00,1.02,1.37,0.68,42.78,0.0,1.55,...,0.0,71.57,0.35,0.00,0.00,0.00,62.60,0.00,0.00,0.0
3,2022-10-22,0.00,0.00,0.00,9.73,1.98,0.00,34.73,0.0,0.00,...,0.0,53.75,0.00,0.00,0.00,0.00,113.32,33.82,0.00,0.0
4,2022-10-24,0.00,6.67,0.00,7.88,12.50,0.43,21.80,0.0,0.35,...,0.0,30.80,19.53,0.00,0.00,0.00,90.28,3.92,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023-05-24,0.12,14.73,0.02,6.35,7.83,1.50,0.00,0.0,0.00,...,0.0,0.00,0.00,1.47,0.00,0.00,40.22,59.38,0.00,0.0
245,2023-05-28,0.00,0.37,0.00,0.13,35.27,0.00,0.00,0.0,0.43,...,0.0,0.00,0.00,0.00,0.00,0.00,33.25,86.02,0.00,0.0
246,2023-05-29,0.00,0.00,0.00,2.42,15.90,1.52,0.00,0.0,0.00,...,0.0,0.00,2.18,0.47,0.00,0.00,42.12,0.00,0.00,0.0
247,2023-05-30,1.63,0.57,0.03,0.55,19.17,4.38,0.00,0.0,3.80,...,0.0,0.00,9.98,0.00,45.28,6.77,100.42,62.63,0.00,0.0
